In [39]:
import math
import torch
import os
import scipy.io  

from torch.utils.data import Dataset, DataLoader
from cnn import wt_net


DATA_PATH = "./DataSets"
MODEL_PATH = "./Models"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 10
EPOCH = 40

# 加载MAT文件  
print("Loading train set...")
label_data = scipy.io.loadmat('./DataSets/SLF/labels.mat')  
sample_data = scipy.io.loadmat('./DataSets/SLF/datas.mat')  



# 将数据重塑为样本和特征的形式
# 每个样本由2行组成，因此总共有500个样本
num_samples = 600
num_features = 4070
samples = torch.from_numpy(sample_data['save_data'])
samples = samples.reshape(num_samples, 1, 2, num_features)
labels = torch.from_numpy(label_data['labels'])
labels = labels.reshape(num_samples)


Loading train set...


In [40]:
print(labels)

tensor([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
         5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,
         6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,
         7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
        11, 11, 11, 11, 12, 12, 12, 12, 

In [41]:

# 创建一个自定义的Dataset类
class CustomDataset(Dataset):
    def __init__(self, samples, labels):
        self.samples = samples
        self.labels = labels

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = torch.tensor(self.samples[idx], dtype=torch.float32)
        print(self.labels)
        label = self.labels[idx]
        return sample, label


# 创建Dataset实例
dataset = CustomDataset(samples, labels)

# 创建DataLoader实例
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
BATCH_NUM = math.ceil(len(train_set) / BATCH_SIZE)
print("Using ", DEVICE)


# 建立模型并载入设备
model = wt_net.MyCNN().to(DEVICE)
# 定义损失及优化器
cost = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

print(
    "\n-----------------\n"
    "Num of epoch: {}\n"
    "Batch size: {}\n"
    "Num of batch: {}".format(EPOCH, BATCH_SIZE, BATCH_NUM)
)
print("-----------------\n")
print("Start training...")

Using  cpu

-----------------
Num of epoch: 40
Batch size: 10
Num of batch: 0
-----------------

Start training...


In [38]:
print(labels.shape)

torch.Size([1, 600])
